In [1]:
import pytorch_lightning as L

In [2]:
from data_loading import load_data, get_loaders, get_probe_data, get_probe_loaders, make_dataset
from transformer_predictor import TransformerPredictor, train, test

In [19]:
dataset = load_data('balanced_with_no_count.csv')
# dataset = load_data('brackets.csv')

In [20]:
BATCH_SIZE = 64
train_loader, val_loader, test_loader, train_data, val_data, test_data = get_loaders(dataset, batch_size=BATCH_SIZE, return_data=True)

In [21]:
print(len(train_data), len(val_data), len(test_data))

24000 4000 12000


In [22]:
model = TransformerPredictor(
    input_dim=4,
    model_dim=128,
    num_classes=2,
    num_heads=2,
    num_layers=1,
    lr=1e-3,
    warmup=100,
    max_iters=1000,
)
trainer = L.Trainer(max_epochs=10, devices=1)

/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home2/adyansh/dinner_pool/rsai/lib/python3.10/site- ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
res,model, trainer, train_outbeddings, val_outbeddings = train(model, trainer, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Sequential         | 640   
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 132 K 
3 | output_net          | Sequential         | 17.0 K
4 | accuracy            | BinaryAccuracy     | 0     
-----------------------------------------------------------
150 K     Trainable params
0         Non-trainable params
150 K     Total params
0.601     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home2/adyansh/dinner_pool/Mechanistic_Interpretability/transformer_predictor.py:280: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  accuracy = self.accuracy(F.softmax(y_hat), y)


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [8]:
# res, train_outbeddings = test(model, trainer, train_loader)
# res, val_outbeddings = test(model, trainer, val_loader)

In [24]:
res, test_outbeddings = test(model, trainer, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9518333077430725     │
│         test_loss         │    0.15723256766796112    │
└───────────────────────────┴───────────────────────────┘

In [25]:
len(train_outbeddings), len(val_outbeddings), len(test_outbeddings)

(3750, 632, 188)

In [26]:
import pandas as pd

no_count_df = pd.read_csv('no_count.csv')
no_count_df.head()

,sequence,stack_depth
0,(())))(()))())((((())((())))((())())()(()))(()...,-2
1,)))()()())))))))()()()(()(())))(())()))((((()(...,0
2,))(()(()())(()))((())))(())()))))(((())(()))()...,0
3,))()))))(())))()))(()(()(())(()))()(())))(()))...,0
4,))()()())())))))(()))()())()()())())))(((())()...,0


In [27]:
from data_loading import BracketDataset
from torch.utils.data import DataLoader

no_count_dataset = BracketDataset(no_count_df)
no_count_loader = DataLoader(no_count_dataset, batch_size=BATCH_SIZE, num_workers=4)

In [28]:
res, no_count_outbeddings = test(model, trainer, no_count_loader)

/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home2/adyansh/dinner_pool/rsai/lib/python3.10/site- ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: |          | 0/? [00:00<?, ?it/s]

/home2/adyansh/dinner_pool/Mechanistic_Interpretability/transformer_predictor.py:280: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  accuracy = self.accuracy(F.softmax(y_hat), y)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5092500448226929     │
│         test_loss         │    1.0128775835037231     │
└───────────────────────────┴───────────────────────────┘

In [11]:
import torch
from torch.utils.data import DataLoader
from data_loading import BracketDataset
from main import Logger

logger = Logger()

def get_stack_depths(loader: DataLoader, stack_depths=[5, 15]):
    # # stack_data = torch.where(torch.isin(data, torch.tensor(stack_depths)))[0]
    # # get the indices of the data that have the stack depth in stack_depths
    # print(len(data))
    # stack_indices = [i for i, x in enumerate(data) if x['sd'] in stack_depths]

    # # if stack_indices is empty, return None
    # if len(stack_indices) == 0:
    #     return None, 0
    
    # # create a new dataset with only the stack depths in stack_depths
    # stack_data = torch.utils.data.Subset(data, stack_indices)
    
    # loader = DataLoader(stack_data, batch_size=BATCH_SIZE)
    
    # return loader, len(stack_indices)
    stack_data = []
    for batch in loader:
        # print(batch)
        for i, x in enumerate(batch['sd']):
            # if type(x) is not dict:
            #     print(x)
            if x in stack_depths:
                sample = {
                    'x': batch['x'][i],
                    'y': batch['y'][i],
                    'sd': x,
                    'eos': batch['eos'][i]
                }
                stack_data.append(sample)
                                
    if len(stack_data) == 0:
        return None, 0
    
    stack_dataset = BracketDataset(list=stack_data)
    
    stack_loader = DataLoader(stack_data, batch_size=BATCH_SIZE)
    
    return stack_loader, len(stack_data)
    


for i in range(10, 101, 10):
    stack_loader, len_stack = get_stack_depths(test_loader, stack_depths=[i])
    if stack_loader is None:
        continue
    res, stack_outbeddings = test(model, trainer, stack_loader)
    logger.log(f"Stack depth {i}, Support: {len_stack}, Accuracy: {res}")
    # print(f"Stack depth {i}, Support: {len_stack}, Accuracy: {res}")
    # break

/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home2/adyansh/dinner_pool/rsai/lib/python3.10/site- ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=39` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

/home2/adyansh/dinner_pool/Mechanistic_Interpretability/transformer_predictor.py:280: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  accuracy = self.accuracy(F.softmax(y_hat), y)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5034722089767456     │
│         test_loss         │    0.6931670308113098     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


...

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.4852941334247589     │
│         test_loss         │    0.6937575340270996     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.4991481900215149     │
│         test_loss         │    0.6933072805404663     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
X_probe_train, y_probe_train = get_probe_data(train_outbeddings, stack_depths=[15, 25])
X_probe_val, y_probe_val = get_probe_data(val_outbeddings, stack_depths=[15, 25])
X_probe_test, y_probe_test = get_probe_data(test_outbeddings, stack_depths=[15, 25])

train_dataset = make_dataset(X_probe_train, y_probe_train)
val_dataset = make_dataset(X_probe_val, y_probe_val)
test_dataset = make_dataset(X_probe_test, y_probe_test)

train_probe_loader, val_probe_loader, test_probe_loader = get_probe_loaders(train_dataset, val_dataset, test_dataset, batch_size=64)

In [ ]:
from probe import Probe, probe_all_models

In [ ]:
probe = Probe(model_name='lr')
# probe.fit(train_probe_loader, val_probe_loader)
# probe.probe(X_probe_train.cpu().detach().numpy(), y_probe_train.cpu().detach().numpy(), X_probe_test.cpu().detach().numpy(), y_probe_test.cpu().detach().numpy())
probe_all_models(X_probe_train.cpu().detach().numpy(), y_probe_train.cpu().detach().numpy(), X_probe_test.cpu().detach().numpy(), y_probe_test.cpu().detach().numpy())

In [ ]:
probe.plot_decision_boundary(X_probe_test.cpu().detach().numpy(), y_probe_test.cpu().detach().numpy())